In [1]:
import dev.biserman.planet.language.Language
import dev.biserman.planet.language.Segment
import dev.biserman.planet.language.SyllableConstructor
import dev.biserman.planet.language.InventoryTransformation
import dev.biserman.planet.language.Manner
import dev.biserman.planet.language.SegmentType


In [2]:
import dev.biserman.planet.language.SegmentData
import java.io.File

SyllableConstructor.languageFile = """E:\Users\Winggar\source\repos\Planet\planet\english.json"""
SyllableConstructor.phonemeFile = """E:\Users\Winggar\source\repos\Planet\planet\phonemes.json"""

val basicTransformation: InventoryTransformation = { inventory ->
    inventory.plus("ptk mn ljw".mapNotNull { SyllableConstructor.segments[it.toString()] })
}

fun (Set<Segment>).addSet(from: (SegmentData) -> Boolean, to: (SegmentData) -> SegmentData): Set<Segment> {
    val new = this
        .filter { from(it.data) }
        .map { to(it.data) }
    val matching = SyllableConstructor.segments.values.filter { it.data in new }
    return this.plus(matching)
}

val inventoryTransformations = listOf<Pair<InventoryTransformation, Int>>(
    { inventory: Set<Segment> ->
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE },
            to = { it.copy(voiced = true) })
    } to 100,
    { inventory: Set<Segment> ->
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE },
            to = { it.copy(manner = Manner.FRICATIVE) })
    } to 100,
)

val test = basicTransformation(setOf())
val test2 = inventoryTransformations[0].first(test)
print(SyllableConstructor.segments.keys)
print(test.map { it.symbol })
test2.map { it.symbol }

[m, n, ŋ, p, t, k, b, d, g, f, θ, ʃ, v, z, ʒ, h, j, w, ɹ, l, i, u, ɪ, e, o, ɛ, ʌ, æ, a][p, t, k, m, n, l, j, w]

[p, t, k, m, n, l, j, w, b, d, g]